In [12]:
from azureml.core import Workspace, Experiment, ScriptRunConfig

ws = Workspace.get(name='quick-starts-ws-135976')
exp = Experiment(workspace=ws, name='udacity-project')

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-135976
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-135976


In [13]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# name for CPU cluster
amlcompute_cluster_name = 'cpu-cluster'

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [18]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--max_iter' : choice(20,40,80,100,150,200),
    '--C' : uniform(0.001,10)
}) 

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2,slack_factor=0.1,delay_evaluation=1)

if "training" not in os.listdir():
    os.mkdir("./training")



# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./",
             compute_target=aml_compute,
             entry_script="train.py")
             

'''
est = ScriptRunConfig(source_directory="./", 
                      script="train.py",
                      compute_target=aml_compute)
'''

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling = ps, 
                                    primary_metric_name = 'Accuracy',
                                    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                    policy=policy,
                                    estimator=est,
                                    max_total_runs=20,
                                    max_concurrent_runs=4)

In [20]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)
assert(hyperdrive_run.get_status()=='Completed')

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_349fefe5-1530-4768-b03f-184da54ad495
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_349fefe5-1530-4768-b03f-184da54ad495?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-135976/workspaces/quick-starts-ws-135976

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-25T11:07:35.746583][API][INFO]Experiment created<END>\n""<START>[2021-01-25T11:07:36.510316][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-01-25T11:07:36.722528][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2021-01-25T11:07:38.1062447Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>

Execution Summary
RunId: HD_349fefe5-1530-4768-b03f-184da54ad495
Web View: https://ml.azure.com/experiments/udacity-project/runs/HD_349fefe5-1530-4768-b03f-184da54ad495?wsid=/subscriptions/b968f

In [35]:
import joblib
# Get your best run and save the model from that run.
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters

# Identify the best performing configuration and save the model from that run. 

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print(parameter_values[0],":", parameter_values[1],"and",parameter_values[2],":",parameter_values[3])
print('Accuracy:', best_run_metrics['Accuracy'])




Best Run Id:  HD_349fefe5-1530-4768-b03f-184da54ad495_0
--C : 0.2132994775663133 and --max_iter : 40
Accuracy: 0.9089529590288316


In [36]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_349fefe5-1530-4768-b03f-184da54ad495_0,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [30]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [31]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [32]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds,
    label_column_name='y',
    n_cross_validations=2)

In [33]:
# Submit your automl run
# https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.experiment(class)?view=azure-ml-py#azureml_core_Experiment_submit

# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-configure-auto-train

automl_run = exp.submit(automl_config, show_output = False)
automl_run.wait_for_completion()

{'runId': 'AutoML_b17da2a0-ae52-4cda-b25b-947e7005ef9e',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2021-01-25T11:30:56.230858Z',
 'endTimeUtc': '2021-01-25T12:03:15.053779Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-135976","workspace_name":"quick-starts-ws-135976","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":2,"y_min":null,"y_max":null,"num_classes":2,"f

In [37]:
# Retrieve and save your best automl model.

# Get the best run object
best_automl_run, fitted_model = automl_run.get_output()

print("Best run metrics :",best_automl_run.get_metrics())

print("Best run details :",best_automl_run.get_details())


Best run metrics : {'AUC_weighted': 0.9461807881772213, 'matthews_correlation': 0.5473359927519399, 'precision_score_micro': 0.9162670713201821, 'recall_score_macro': 0.751072413733951, 'accuracy': 0.9162670713201821, 'AUC_micro': 0.9804091544414791, 'precision_score_weighted': 0.910589444001151, 'f1_score_weighted': 0.9126710743403259, 'AUC_macro': 0.9461807881772213, 'average_precision_score_weighted': 0.9546541916196508, 'average_precision_score_macro': 0.8224103434997676, 'norm_macro_recall': 0.5021448274679019, 'f1_score_macro': 0.7717598059839932, 'balanced_accuracy': 0.751072413733951, 'recall_score_micro': 0.9162670713201821, 'weighted_accuracy': 0.9573020400613186, 'average_precision_score_micro': 0.9811064617654615, 'f1_score_micro': 0.9162670713201821, 'recall_score_weighted': 0.9162670713201821, 'log_loss': 0.19194319169742724, 'precision_score_macro': 0.7983850702909261, 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_b17da2a0-ae52-4cda-b25b-947e7005ef9e_47/a

In [38]:
best_automl_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_b17da2a0-ae52-4cda-b25b-947e7005ef9e_47,,Completed,Link to Azure Machine Learning studio,Link to Documentation
